In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train=pd.read_csv("train (2).csv")
df_test=pd.read_csv("test (2).csv")

In [3]:
valid_cat = ['MiscFeature','Fence','PoolQC','GarageCond','GarageQual','GarageFinish','GarageType','FireplaceQu','BsmtFinType2', 'BsmtFinType1','BsmtExposure','BsmtCond','BsmtQual','Alley']
NA_missing=set(df_test.columns)-set(valid_cat)
d1=dict.fromkeys(valid_cat,[''])
d2=dict.fromkeys(NA_missing,['','NA'])
d1.update(d2)



In [4]:
df_train=pd.read_csv("train (2).csv",keep_default_na=False,na_values=d1)
df_test=pd.read_csv("test (2).csv",keep_default_na=False,na_values=d1)

In [5]:
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NA,Reg,Lvl,AllPub,...,0,NA,MnPrv,NA,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NA,Reg,Lvl,AllPub,...,0,NA,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,4,2010,WD,Normal,142125


In [6]:
df_train.shape

(1460, 81)

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [8]:
valid_cat = ['MiscFeature','Fence','PoolQC','GarageCond','GarageQual','GarageFinish','GarageType','FireplaceQu','BsmtFintypeZ', 'BsmtFinType1','BsmtExposure','BsmtCond','BsmtQual','Alley']

In [9]:
df_train['Alley'].value_counts()

Alley
NA      1369
Grvl      50
Pave      41
Name: count, dtype: int64

In [10]:
df_train['LotFrontage'].value_counts()

LotFrontage
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
137.0      1
141.0      1
38.0       1
140.0      1
46.0       1
Name: count, Length: 110, dtype: int64

In [11]:
column_cat=list(df_test.select_dtypes(include=object).columns)
column_num=list(df_test.select_dtypes(exclude=object).columns)

Missing values handling


In [13]:
df_train.isna().sum().sum()

357

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
imputer_num=SimpleImputer(strategy='median')
imputer_num.fit(df_train[column_num])# median of columns_num
df_train[column_num]=imputer_num.transform(df_train[column_num])

imputer_cat=SimpleImputer(strategy='most_frequent')
imputer_cat.fit(df_train[column_cat])# mode of columns_cat
df_train[column_cat]=imputer_cat.transform(df_train[column_cat])

onehot encoding


In [17]:
from sklearn.preprocessing import OneHotEncoder
import warnings


In [18]:
encoder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
encoder.fit(df_train[column_cat])#new columns name
columns_enc=list(encoder.get_feature_names_out())
df_train[columns_enc]=encoder.transform(df_train[column_cat])

Feature Scaling

In [20]:
from sklearn.preprocessing import MinMaxScaler

In [21]:
scaler=MinMaxScaler()
scaler.fit(df_train[column_num])
df_train[column_num]=scaler.transform(df_train[column_num])

Model Building

In [23]:
X_train=df_train[column_num+columns_enc]
y_train=df_train['SalePrice']

In [24]:
from sklearn.linear_model import LinearRegression

In [25]:
model=LinearRegression()
model.fit(X_train,y_train)
model.score(X_train,y_train)

0.9332632625064228

Make prediction on test data

In [27]:
df_test[column_num]=imputer_num.transform(df_test[column_num])
df_test[column_cat]=imputer_cat.transform(df_test[column_cat])
df_test[columns_enc]=encoder.transform(df_test[column_cat])
df_test[column_num]=scaler.transform(df_test[column_num])
X_test=df_test[column_num+columns_enc]
yp=model.predict(X_test)

In [28]:
df_submit=pd.DataFrame({'id':range(1461,2920),'SalePrice':yp})
df_submit.to_csv('sub2.csv',index=False)

In [29]:
yp

array([113236., 161348., 187548., ..., 181616., 117784., 225692.])